In [1]:
try:
    !pip install pyspark=="2.4.5" --quiet
    !pip install pandas=="1.0.4" --quiet
except:
    print("Running throw py file.")

In [2]:
from pyspark import SparkContext as sc
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import Window
from pyspark import SparkFiles
from pyspark.sql.types import StringType
import pyspark
import json
import pandas as pd
import time

In [3]:
spark = SparkSession\
        .builder\
        .appName("Desafio Data Engineer Escale - Fabio Kfouri")\
        .getOrCreate()
spark

In [4]:
import os

dataPath = 'https://d3l36jjwr70u5l.cloudfront.net/data-engineer-test/'

if 'E:\\' in os.getcwd() and 'dataEngineerTest_Escale' in os.getcwd():
    dataPath = os.getcwd() + "/data/"

print(dataPath)


E:\Projetos\Jobs\dataEngineerTest_Escale/data/


In [5]:
for i in range(9):
    spark.sparkContext.addFile(dataPath + 'part-0000' + str(i) +'.json.gz')

Py4JJavaError: An error occurred while calling o35.addFile.
: java.nio.file.FileSystemException: E:\Projetos\Jobs\dataEngineerTest_Escale\data\part-00005.json.gz -> C:\Users\fkfouri\AppData\Local\Temp\spark-158396a6-2fae-44ae-8e16-d306acb33621\userFiles-77782f07-9868-4103-b37e-7f5f2ff8502b\part-00005.json.gz: There is not enough space on the disk.

	at sun.nio.fs.WindowsException.translateToIOException(WindowsException.java:86)
	at sun.nio.fs.WindowsException.rethrowAsIOException(WindowsException.java:97)
	at sun.nio.fs.WindowsFileCopy.copy(WindowsFileCopy.java:205)
	at sun.nio.fs.WindowsFileSystemProvider.copy(WindowsFileSystemProvider.java:278)
	at java.nio.file.Files.copy(Files.java:1274)
	at org.apache.spark.util.Utils$.org$apache$spark$util$Utils$$copyRecursive(Utils.scala:664)
	at org.apache.spark.util.Utils$.copyFile(Utils.scala:635)
	at org.apache.spark.util.Utils$.doFetchFile(Utils.scala:719)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:509)
	at org.apache

In [ ]:
def readFile(i):
    return spark.read.json(SparkFiles.get('part-0000' + str(i) +'.json.gz'))


In [ ]:
df0 = readFile(0)
df0.show(3)

In [ ]:
df0.count()

In [ ]:
df0.printSchema()

In [ ]:
#
df = df0.groupBy('anonymous_id').count().sort(F.col("count").desc())
df.show(10,False)

In [ ]:
df = df0.filter('anonymous_id = "ae59b395-31ac-40e3-b7c0-3d97d00ac6cc"').sort(F.col("device_sent_timestamp"))
df.show()

In [ ]:
overCategory = Window.partitionBy('anonymous_id','device_family','os_family').orderBy('device_sent_timestamp')

In [ ]:
dftemp = df.withColumn("lag", F.lag('device_sent_timestamp', 1).over(overCategory))

In [ ]:
time_limit = (30*60)*1000
print(time_limit)

In [ ]:
dftemp = dftemp.withColumn('delta', F.col('device_sent_timestamp') - F.col('lag'))\
        .withColumn('same_section', (F.col('device_sent_timestamp') - F.col('lag')) < time_limit) \
        .withColumn('event_time', (F.col('device_sent_timestamp')/1000).cast('timestamp'))

dftemp.show()

In [ ]:
dftemp.coalesce(1).write.format("json").save("analise.json")

In [ ]:
dftemp = spark.read.json(os.getcwd() + '/analise.json')

In [ ]:
dftemp.createOrReplaceTempView("clickstream")

In [ ]:
df_question_1 = spark.sql("""
with temp as (--
      SELECT anonymous_id, browser_family, delta, device_family, device_sent_timestamp, event, event_time, n, os_family, platform, nvl(same_section,false) same_section, version
        FROM clickstream t --
        where nvl(same_section, false) = false
)
select temp.*, 'session_' || ROW_NUMBER() OVER (PARTITION BY anonymous_id ORDER BY device_sent_timestamp )  || '_' ||  anonymous_id as session_id
  from temp
""")
df_question_1.show(300, False)

In [ ]:
df_question_1 = spark.sql("""
with table_temp as (--
    SELECT anonymous_id, browser_family, delta, device_family, device_sent_timestamp, event, event_time, n, os_family, platform, NVL(same_section,false) same_section, version
    FROM clickstream t --
    WHERE NVL(same_section, false) = false --remove as sessoes abertas
), table_session_id as (--
    SELECT t.*, 'session_' || ROW_NUMBER() OVER (PARTITION BY anonymous_id ORDER BY device_sent_timestamp )  || '_' ||  anonymous_id as session_id
    FROM table_temp t --
)
SELECT COUNT(session_id)  qtd_session
FROM table_session_id
""")
df_question_1.show(300, False)

In [ ]:
x = df_question_1.collect()[0]['qtd_session']
x